## Disaster Tweets Classification with AutoKeras
In this notebook, I will build a Disaster Tweets Classifier using autokeras. You can learn more about it from https://autokeras.com.

## Setup

In [ ]:
!pip -q install autokeras

In [ ]:
import autokeras
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

## Import datatset

In [ ]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test.head()

## Modeling

In [ ]:
batch_size = 64
train_ds = tf.data.Dataset.from_tensor_slices((train["text"], train["target"])).batch(batch_size).cache()
classifier = autokeras.TextClassifier(
    num_classes=2,
    max_trials=1,
    objective="val_loss",
    directory="tweet",
    seed=42
)
classifier.fit(train_ds, epochs=10, validation_split=0.2)

## Export best Model

In [ ]:
model = classifier.export_model()
model.save("model.h5")

## Load Model

In [ ]:
loaded_model = load_model("model", custom_objects=autokeras.CUSTOM_OBJECTS)

## Submission

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test["text"])).batch(batch_size)
result = np.array(loaded_model.predict(test_ds) > 0.5, dtype=int).reshape(-1)
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission["target"] = result
submission.to_csv("submission.csv", index=False)
submission.head()